# Salary prediction Project
1. Defining the problem statement (What objectives is the solution trying to solve)
2. Discover( We will obtain data, clean data, explore data, establish baseline outcomes and hypothesize the solution)
3. Developing (Engineer features, create models, test models, and select bes model)
4. Deploying (automate the pipeline, deploy solution, measure efficacy- how much are we positively affecting the KPI)

In [1]:
#importing all libraries we are going to use
import pandas as pd
import numpy as np
import seaborn as sns

from matplotlib import *
import sys
import pylab as pl
import pickle
% matplotlib inline

ImportError: cannot import name 'remove_na'

In [61]:
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.externals import joblib
from sklearn import svm
from sklearn.neighbors import KNeighborsRegressor

In [3]:
#import data
def import_data(file):
    return pd.read_csv(file)

In [4]:
train_features_df = import_data("train_features.csv")
test_features_df = import_data("test_features.csv")
train_salaries_df = import_data("train_salaries.csv")

In [5]:
#head function by default will return the first 5 rows, but you can explicitly specify any number of rows
testing = join_data(train_features_df,train_salaries_df)
testing.tail(10)

NameError: name 'join_data' is not defined

In [6]:
test_features_df.head(10)

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis
0,JOB1362685407687,COMP33,MANAGER,HIGH_SCHOOL,NONE,HEALTH,22,73
1,JOB1362685407688,COMP13,JUNIOR,NONE,NONE,AUTO,20,47
2,JOB1362685407689,COMP10,CTO,MASTERS,BIOLOGY,HEALTH,17,9
3,JOB1362685407690,COMP21,MANAGER,HIGH_SCHOOL,NONE,OIL,14,96
4,JOB1362685407691,COMP36,JUNIOR,DOCTORAL,BIOLOGY,OIL,10,44
5,JOB1362685407692,COMP40,CTO,MASTERS,COMPSCI,FINANCE,6,23
6,JOB1362685407693,COMP32,SENIOR,MASTERS,COMPSCI,SERVICE,6,32
7,JOB1362685407694,COMP11,CEO,BACHELORS,BIOLOGY,SERVICE,9,73
8,JOB1362685407695,COMP39,MANAGER,DOCTORAL,PHYSICS,HEALTH,6,78
9,JOB1362685407696,COMP22,VICE_PRESIDENT,NONE,NONE,AUTO,23,97


In [7]:
#this is our target variable(salary), we will be predicting against this variable
train_salaries_df.head()

,jobId,salary
0,JOB1362684407687,130
1,JOB1362684407688,101
2,JOB1362684407689,137
3,JOB1362684407690,142
4,JOB1362684407691,163


In [8]:
# check features information and the data types for each feature
train_features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   jobId                1000000 non-null  object
 1   companyId            1000000 non-null  object
 2   jobType              1000000 non-null  object
 3   degree               1000000 non-null  object
 4   major                1000000 non-null  object
 5   industry             1000000 non-null  object
 6   yearsExperience      1000000 non-null  int64 
 7   milesFromMetropolis  1000000 non-null  int64 
dtypes: int64(2), object(6)
memory usage: 61.0+ MB


In [9]:
test_features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   jobId                1000000 non-null  object
 1   companyId            1000000 non-null  object
 2   jobType              1000000 non-null  object
 3   degree               1000000 non-null  object
 4   major                1000000 non-null  object
 5   industry             1000000 non-null  object
 6   yearsExperience      1000000 non-null  int64 
 7   milesFromMetropolis  1000000 non-null  int64 
dtypes: int64(2), object(6)
memory usage: 61.0+ MB


In [10]:
train_salaries_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   jobId   1000000 non-null  object
 1   salary  1000000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 15.3+ MB


In [11]:
# helper function to check for duplicates
def check_duplicates(data):
    return data.duplicated().sum()

In [12]:
check_duplicates(train_features_df)

0

In [13]:
check_duplicates(test_features_df)

0

In [14]:
check_duplicates(train_salaries_df)

0

In [15]:
# spliting numeric and categorical features
numeric_cols = ['yearsExperience','milesFromMetropolis']

In [16]:
categorical_cols = ['jobId','companyId','jobType','degree','major','industry']

In [17]:
train_features_df.describe(include = [np.number])

,yearsExperience,milesFromMetropolis
count,1000000.000000,1000000.000000
mean,11.992386,49.529260
std,7.212391,28.877733
min,0.000000,0.000000
25%,6.000000,25.000000
50%,12.000000,50.000000
75%,18.000000,75.000000
max,24.000000,99.000000


In [18]:
train_features_df.describe(include = ['O'])

,jobId,companyId,jobType,degree,major,industry
count,1000000,1000000,1000000,1000000,1000000,1000000
unique,1000000,63,8,5,9,7
top,JOB1362685264351,COMP39,SENIOR,HIGH_SCHOOL,NONE,WEB
freq,1,16193,125886,236976,532355,143206


In [19]:
#combine the train feature with the target feature on job id
train_df = pd.merge(train_features_df, train_salaries_df, on='jobId')

In [20]:
#optionally delete the original data set
del train_features_df

In [21]:
del train_salaries_df

In [22]:
#we can see the combined data with 1000 000 rows
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 999999
Data columns (total 9 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   jobId                1000000 non-null  object
 1   companyId            1000000 non-null  object
 2   jobType              1000000 non-null  object
 3   degree               1000000 non-null  object
 4   major                1000000 non-null  object
 5   industry             1000000 non-null  object
 6   yearsExperience      1000000 non-null  int64 
 7   milesFromMetropolis  1000000 non-null  int64 
 8   salary               1000000 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 76.3+ MB


In [23]:
#checking the first five entries, you can also use tail to check the last five entries
train_df.head()

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary
0,JOB1362684407687,COMP37,CFO,MASTERS,MATH,HEALTH,10,83,130
1,JOB1362684407688,COMP19,CEO,HIGH_SCHOOL,NONE,WEB,3,73,101
2,JOB1362684407689,COMP52,VICE_PRESIDENT,DOCTORAL,PHYSICS,HEALTH,10,38,137
3,JOB1362684407690,COMP38,MANAGER,DOCTORAL,CHEMISTRY,AUTO,8,17,142
4,JOB1362684407691,COMP7,VICE_PRESIDENT,BACHELORS,PHYSICS,FINANCE,8,16,163


In [24]:
# Using the box and distribution plots to check for outliers
pl.figure(figsize = (14, 6))
pl.subplot(1,2,1)
sns.boxplot(train_df.salary)
pl.subplot(1,2,2)
sns.distplot(train_df.salary, bins=20)
pl.show()

NameError: name 'pl' is not defined

### What does the graph tells us
1. outliers suspected on 0< salaries and >220 salaries 

In [25]:
stat = train_df.salary.describe()
print(stat)
IQR = stat['75%'] - stat['25%']
upper = stat['75%'] + 1.5 * IQR
lower = stat['25%'] - 1.5 * IQR
print('The upper and lower bounds for suspected outliers are {} and {}'.format(upper, lower))

count    1000000.000000
mean         116.061818
std           38.717936
min            0.000000
25%           88.000000
50%          114.000000
75%          141.000000
max          301.000000
Name: salary, dtype: float64
The upper and lower bounds for suspected outliers are 220.5 and 8.5


In [26]:
# display information on the lower bounds outliers
train_df[train_df.salary < 8.5]

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary
30559,JOB1362684438246,COMP44,JUNIOR,DOCTORAL,MATH,AUTO,11,7,0
495984,JOB1362684903671,COMP34,JUNIOR,NONE,NONE,OIL,1,25,0
652076,JOB1362685059763,COMP25,CTO,HIGH_SCHOOL,NONE,AUTO,6,60,0
816129,JOB1362685223816,COMP42,MANAGER,DOCTORAL,ENGINEERING,FINANCE,18,6,0
828156,JOB1362685235843,COMP40,VICE_PRESIDENT,MASTERS,ENGINEERING,WEB,3,29,0


In [27]:
#check job types of these high earners 
train_df.loc[train_df.salary > 220.5, 'jobType'].value_counts()

CEO               3227
CFO               1496
CTO               1488
VICE_PRESIDENT     603
MANAGER            217
SENIOR              66
JUNIOR              20
Name: jobType, dtype: int64

In [28]:
# from above, we observed job type "Junior" being among the high earners. we will investigate further
train_df[(train_df.salary > 220.5) & (train_df.jobType == 'JUNIOR')]

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary
1222,JOB1362684408909,COMP40,JUNIOR,MASTERS,COMPSCI,OIL,24,5,225
27710,JOB1362684435397,COMP21,JUNIOR,DOCTORAL,ENGINEERING,OIL,24,3,246
31355,JOB1362684439042,COMP45,JUNIOR,DOCTORAL,COMPSCI,FINANCE,24,0,225
100042,JOB1362684507729,COMP17,JUNIOR,DOCTORAL,BUSINESS,FINANCE,23,8,248
160333,JOB1362684568020,COMP18,JUNIOR,DOCTORAL,BUSINESS,FINANCE,22,3,223
189582,JOB1362684597269,COMP32,JUNIOR,DOCTORAL,BUSINESS,OIL,24,11,221
214606,JOB1362684622293,COMP47,JUNIOR,MASTERS,BUSINESS,FINANCE,22,4,222
303778,JOB1362684711465,COMP51,JUNIOR,MASTERS,ENGINEERING,WEB,24,2,226
348354,JOB1362684756041,COMP56,JUNIOR,DOCTORAL,ENGINEERING,OIL,23,25,226
427593,JOB1362684835280,COMP54,JUNIOR,DOCTORAL,ENGINEERING,FINANCE,23,3,221


In [29]:
# looking at the years of experience, level of education and industry we can see these are valid earnings
train_df = train_df[train_df.salary > 8.5]

In [30]:
# this helper function will take the dataframe and the column to plot against
def plot_feature(df, col):
    pl.figure(figsize = (16,6))
    pl.subplot(1,2,1)
    if  df[col].dtype == 'int64': #if the data type is numerics, we will plot the counts
        df[col].value_counts().plot()
    else:
        mean = df.groupby(col)['salary'].mean()
        df[col] = df[col].astype('category') # column with be of type categorical
        levels = mean.sort_values().index.tolist() 
        df[col].cat.reorder_categories(levels, inplace=True)
        df[col].value_counts().plot()
    pl.xticks(rotation=45)
    pl.xlabel(col)
    pl.ylabel('Counts')
    pl.subplot(1,2,2)
    
    if df[col].dtype == 'int64' or col == 'companyId':
        mean = df.groupby(col)['salary'].mean()
        std = df.groupby(col)['salary'].std()
        mean.plot()
        pl.fill_between(range(len(std.index)), mean.values-std.values, mean.values + std.values, \
                       alpha = 0.1)
    else:
        sns.boxplot(x = col, y = 'salary', data = df)
    
    pl.xticks(rotation=45)
    pl.ylabel('salaries')
    pl.show()

In [31]:
plot_feature(train_df,'companyId')

NameError: name 'pl' is not defined

In [32]:
plot_feature(train_df, 'jobType')

NameError: name 'pl' is not defined

In [33]:
plot_feature(train_df, 'degree')

NameError: name 'pl' is not defined

In [34]:
plot_feature(train_df, 'major')

NameError: name 'pl' is not defined

In [35]:
plot_feature(train_df, 'industry')

NameError: name 'pl' is not defined

In [36]:
plot_feature(train_df, 'yearsExperience')

NameError: name 'pl' is not defined

In [37]:
plot_feature(train_df, 'milesFromMetropolis')

NameError: name 'pl' is not defined

In [38]:
def encode_label(df, col):
    cat_dict ={}
    cats = df[col].cat.categories.tolist()
    for cat in cats:
        cat_dict[cat] = train_df[train_df[col] == cat]['salary'].mean()
    df[col] = df[col].map(cat_dict)
    

In [39]:
for col in train_df.columns:
    if train_df[col].dtype.name == "category":
       encode_label(train_df, col) 

In [40]:
fig = pl.figure(figsize=(12,10))
features = ['companyId','jobType','degree','major','industry','yearsExperience','milesFromMetropolis']
sns.heatmap(train_df[features + ['salary']].corr(), cmap='Blues', annot=True)
pl.xticks(rotation=45)
pl.show()

NameError: name 'pl' is not defined

In [41]:
#join feature and target data
def join_data(df1, df2, key=None, left_index=False, right_index=False):
    return pd.merge(df1, df2, how='inner', on=key, left_index=left_index, right_index=right_index)

In [42]:
#remove duplicates and any rows with 0 salary
def clean(data):
    clean_data = data.drop_duplicates(subset='jobId')
    clean_data = clean_data[clean_data.salary>0]
    return clean_data

In [43]:
# encode all features to numeric values
def one_hot_encode_feature_df(df,cat_vars=None, num_vars=None):
    #get dummies function will turn our categorical feature into numeric  
    # as some models may not be able to deal with categorical features
    cat_df = pd.get_dummies(df, columns=cat_vars, dummy_na=True)
    # we just want to enforce the numerical features to return numeric values
    num_df = df[num_vars].apply(pd.to_numeric) 
    return pd.concat([cat_df,num_df], axis=1) # combine all features together

In [44]:
def ohe(df, cat_vars=None, num_vars=None):
    cat_df = pd.get_dummies(df, prefix_sep="__", columns=cat_vars)
    #num_df = df[num_vars].apply(pd.to_numeric) 
    return cat_df

In [45]:
# return a dataframe with only target feature
def get_target_df(df, target):
    return df[target]

In [80]:
#pass the model and feature and target dataframes, calculate the mse,std and return a list of dictionary
def train_model(models, feature_df,target_df, num_procs):
    results = []
    for model in models:
        neg_mse = cross_val_score(model, feature_df,target_df, cv=5, n_jobs=num_procs, scoring='neg_mean_squared_error')
        mean_mse = -1.0*np.mean(neg_mse)
        cv_std = np.std(neg_mse)  
        results.append({'model':model,'MSE':mean_mse ,'CV':cv_std})
       
    return results

In [47]:
#display model information
def print_summary(model):
    print('\nModel:\n', model['model'])
    print('Average MSE:\n', model['MSE'])
    print('Standard deviation during CV:\n', model['CV'])

In [48]:
# save predictions, feature importances to flat files
def save_results(model, mean_mse, predictions, feature_importance):
    with open('model.txt', 'w') as file:
        file.write(str(model))
    feature_importance.to_csv('feature_importance.csv')
    np.savetxt('predictions.csv', predictions, delimiter=',')

In [49]:
def save_model(model):
     #save model to disk
    pickle.dump(model, open('model.pkl','wb'))
    # load the model to compare the results
    model = pickle.load(open('model.pkl','rb'))
    

In [50]:
def get_categories(df_):
    categoricals = []
    for col, col_type in df_.dtypes.iteritems():
        if col_type == 'O':
            categoricals.append(col)
        else:
            df_[col].fillna(0, inplace=True)
    return categoricals       
            

In [51]:
# main application

#training data
train_features_file = 'train_features.csv'
test_features_file = 'test_features.csv'
train_salaries_file = 'train_salaries.csv'

#seperate categorical and numeric features 
cat_vars = ['jobType','degree','major','industry']
num_vars = ['yearsExperience','milesFromMetropolis']
target_var = 'salary'

all_col = ['jobType','degree','major','industry','yearsExperience','milesFromMetropolis']
#use helper functions to load the data
print('Loading data...')
features_df = import_data(train_features_file)
test_df = import_data(test_features_file)
target_df = import_data(train_salaries_file)
new_df = join_data(features_df,target_df)
new_df = clean(new_df)
target_df = clean(target_df)
df = pd.DataFrame(columns= ['jobId', 'companyId','jobType','degree','major','industry','yearsExperience','milesFromMetropolis'])
jobId = "JOB1362685407687"
companyId = "COMP33"
jobType = "JUNIOR"
degree = "MASTERS"
major = "COMPSCI"
industry = "OIL"
yearsExperience = 24
milesFromMetropolis = 5


df1 = pd.DataFrame(data=[[jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis]],columns=['jobId', 'companyId','jobType','degree','major','industry','yearsExperience','milesFromMetropolis'])
features_df = new_df[all_col]
test_df = test_df[all_col]


Loading data...


In [1]:
ohe_df = ohe(features_df,cat_vars=cat_vars)
ohe_df_test = ohe(test_df, cat_vars=cat_vars)
df_ =  ohe(df, cat_vars=cat_vars)

#ohe_df.drop(ohe_df.columns[[0,1]], axis=1, inplace=True)
#ohe_df_test.drop(ohe_df_test.columns[[0,1]], axis=1, inplace=True)
#df_.drop(df_.columns[[0,1]], axis=1, inplace=True)
target_df

NameError: name 'ohe' is not defined

In [53]:
from sklearn.model_selection import train_test_split
y = target_df.salary
X_train, X_test, y_train, y_test = train_test_split(ohe_df, y, test_size=0.2)

In [54]:
def ohe(df, cat_vars):
    df_processed = pd.get_dummies(df, prefix_sep="__", columns=cat_vars)
    return df_processed

In [55]:
def col_dummies(df_processed):
    cat_dummies = [col for col in df_processed if "__" in col and col.split("__")[0] in cat_vars]
    return cat_dummies

In [56]:
cat_dummies = col_dummies(ohe_df)
df_processed_columns = list(ohe_df.columns[:])

In [57]:
def rem_addition_cols(df_test_processed,cat_vars,cat_dummies,df_processed_columns):
    for col in df_test_processed.columns:
        if("__" in col) and (col.split("__"[0]) in cat_vars) and col not in cat_dummies:
            print("Removing additional feature {}".format(col))
            df_test_processed.drop(col, axis=1, inplace = True)
        else:
            print("Nothing to remove")

    for col in cat_dummies:
        if col not in df_test_processed.columns and col != ['salary']:
            print("Adding missing feature {}".format(col))
            df_test_processed[col] = 0
    df = df_test_processed[df_processed_columns]
    return df

In [58]:
df_ = rem_addition_cols(test_df,cat_vars,cat_dummies,df_processed_columns)

Nothing to remove
Nothing to remove
Nothing to remove
Nothing to remove
Nothing to remove
Nothing to remove
Adding missing feature jobType__CEO
Adding missing feature jobType__CFO
Adding missing feature jobType__CTO
Adding missing feature jobType__JANITOR
Adding missing feature jobType__JUNIOR
Adding missing feature jobType__MANAGER
Adding missing feature jobType__SENIOR
Adding missing feature jobType__VICE_PRESIDENT
Adding missing feature degree__BACHELORS
Adding missing feature degree__DOCTORAL
Adding missing feature degree__HIGH_SCHOOL
Adding missing feature degree__MASTERS
Adding missing feature degree__NONE
Adding missing feature major__BIOLOGY
Adding missing feature major__BUSINESS
Adding missing feature major__CHEMISTRY
Adding missing feature major__COMPSCI
Adding missing feature major__ENGINEERING
Adding missing feature major__LITERATURE
Adding missing feature major__MATH
Adding missing feature major__NONE
Adding missing feature major__PHYSICS
Adding missing feature industry__A

In [81]:
 #merge the training feature with the target feature, using the helper functions
#raw_train_df = join_data(features_df, target_df, key='jobId')
# clean the data, remove duplicates and samples with 0 salaries
#clean_train_df = shuffle(clean(ohe_df)).reset_index()

print('Encoding data...')
# Encode the features into numeric
#features_df = one_hot_encode_feature_df(clean_train_df, cat_vars=cat_vars, num_vars= num_vars)
#test_df = one_hot_encode_feature_df(test_df, cat_vars=cat_vars, num_vars= num_vars)
# from the full data set, we want to extract the target feature
#target_df = get_target_df(clean_train_df,target_var)

#we will append all models we want to train on into a list
models = []

num_procs=5

verbose_lvl = 0

#list of models we want to choose from to make our predictions
lr= LinearRegression()
lr_std_pca= make_pipeline(StandardScaler, PCA(), LinearRegression())
rf = RandomForestRegressor(n_jobs=num_procs, max_depth=25, min_samples_split=60, \
                           max_features=20, verbose=verbose_lvl)
gbn = GradientBoostingRegressor(max_depth=5, loss='ls', verbose=verbose_lvl)
dct = DecisionTreeRegressor(random_state=1)
knn = KNeighborsRegressor()
svm = svm.SVR()
# add the above models into a list to be iterated through
models.extend([lr,rf,gbn,knn])
#del ohe_df['jobId']
# begining of our model validation
print('Begin cross validation...')
# results will hold list of dictionary with models validations data
results = train_model(models, X_train, y_train, num_procs)
print('Cross validation complete.')

Encoding data...
Begin cross validation...
Cross validation complete.


In [82]:
# get the minimum Mean Square error value of all the models
min_ = min(results, key=lambda x:x['MSE'])

In [83]:
# get the model associated with the min mse
model = min_['model']
# display summary information of the selected model

In [84]:
for m in results:
    print_summary(m)


Model:
 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
Average MSE:
 384.4566652281443
Standard deviation during CV:
 1.221356684472089

Model:
 RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=25,
           max_features=20, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=60, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=5, oob_score=False, random_state=None,
           verbose=0, warm_start=False)
Average MSE:
 369.01593281008184
Standard deviation during CV:
 1.4425056908900629

Model:
 GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=5, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimat

In [85]:
model.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=5, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [86]:
model.predict(X_test)

array([ 73.07698578, 131.50295774, 150.8653814 , ..., 177.29524   ,
       106.11510561,  49.77377718])

In [87]:
save_model(model)

In [88]:
model.score(X_test,y_test)

0.759929750208316

In [72]:
import pickle
#save model to disk
# load the model to compare the results
model_plk = pickle.load(open('model.pkl','rb'))
dfp = pd.read_pickle('model.pkl')


In [73]:
def save_colums(df_processed_columns, cat_dummies):
     #save model to disk
    processed_columns = open('processed_columns','wb')
    pickle.dump(df_processed_columns,processed_columns)
    processed_columns.close()
    
    cat_dummies_ = open('cat_dummies','wb')
    pickle.dump(cat_dummies,cat_dummies_)
    cat_dummies_.close()
    

In [74]:
save_colums(df_processed_columns,cat_dummies)

In [75]:
infile_processed_columns = open('processed_columns','rb')
new_dict_processed_columns = pickle.load(infile_processed_columns)
infile_processed_columns.close()

infile_cat_dummies = open('cat_dummies','rb')
new_dict_cat_dummies = pickle.load(infile_cat_dummies)
infile_cat_dummies.close()
new_dict_cat_dummies

['jobType__CEO',
 'jobType__CFO',
 'jobType__CTO',
 'jobType__JANITOR',
 'jobType__JUNIOR',
 'jobType__MANAGER',
 'jobType__SENIOR',
 'jobType__VICE_PRESIDENT',
 'degree__BACHELORS',
 'degree__DOCTORAL',
 'degree__HIGH_SCHOOL',
 'degree__MASTERS',
 'degree__NONE',
 'major__BIOLOGY',
 'major__BUSINESS',
 'major__CHEMISTRY',
 'major__COMPSCI',
 'major__ENGINEERING',
 'major__LITERATURE',
 'major__MATH',
 'major__NONE',
 'major__PHYSICS',
 'industry__AUTO',
 'industry__EDUCATION',
 'industry__FINANCE',
 'industry__HEALTH',
 'industry__OIL',
 'industry__SERVICE',
 'industry__WEB']

In [76]:
df_new = pd.DataFrame(columns= ['jobId', 'companyId','jobType','degree','major','industry','yearsExperience','milesFromMetropolis'])
jobId = "JOB1362685407680"
companyId = "COMP21"
jobType = "CFO"
degree = "MASTERS"
major = "CHEMISTRY"
industry = "SERVICE"
yearsExperience = 4
milesFromMetropolis = 6


df1 = pd.DataFrame(data=[[jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis]],columns=['jobId', 'companyId','jobType','degree','major','industry','yearsExperience','milesFromMetropolis'])
df_new = pd.concat([df_new,df1], axis=0)
df_new = ohe(df_new,cat_vars)

In [77]:
df_ = rem_addition_cols(df_new,cat_vars,new_dict_cat_dummies,new_dict_processed_columns)

Nothing to remove
Nothing to remove
Nothing to remove
Nothing to remove
Nothing to remove
Nothing to remove
Nothing to remove
Nothing to remove
Adding missing feature jobType__CEO
Adding missing feature jobType__CTO
Adding missing feature jobType__JANITOR
Adding missing feature jobType__JUNIOR
Adding missing feature jobType__MANAGER
Adding missing feature jobType__SENIOR
Adding missing feature jobType__VICE_PRESIDENT
Adding missing feature degree__BACHELORS
Adding missing feature degree__DOCTORAL
Adding missing feature degree__HIGH_SCHOOL
Adding missing feature degree__NONE
Adding missing feature major__BIOLOGY
Adding missing feature major__BUSINESS
Adding missing feature major__COMPSCI
Adding missing feature major__ENGINEERING
Adding missing feature major__LITERATURE
Adding missing feature major__MATH
Adding missing feature major__NONE
Adding missing feature major__PHYSICS
Adding missing feature industry__AUTO
Adding missing feature industry__EDUCATION
Adding missing feature industry_

In [341]:
df_

,yearsExperience,milesFromMetropolis,jobType__CEO,jobType__CFO,jobType__CTO,jobType__JANITOR,jobType__JUNIOR,jobType__MANAGER,jobType__SENIOR,jobType__VICE_PRESIDENT,...,major__MATH,major__NONE,major__PHYSICS,industry__AUTO,industry__EDUCATION,industry__FINANCE,industry__HEALTH,industry__OIL,industry__SERVICE,industry__WEB
0,4,6,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [78]:
prediction = model_plk.predict(df_)

In [79]:
prediction

array([123.61340565])

In [244]:
#get feature inmportances from the model selected, add to a dataframe and plot the graph
if hasattr(model, 'feature_importances_'):
    importances = model.feature_importances_
        
else:
    importances = [0]*len(features_df.columns)
        
feature_importances = pd.DataFrame({'feature':features_df.columns, 'importance':importances})
feature_importances.sort_values(by='importance', ascending=False, inplace=True)


feature_importances.set_index('feature', inplace=True, drop=True)
# save the predicted and feature importance results into seperate flat files
save_results(model, mean_mse[model], predictions, feature_importances)


ValueError: arrays must all be same length

In [245]:
#plot the feature importance results
feature_importances[0:25].plot.bar(figsize=(20,10))
pl.show()

NameError: name 'feature_importances' is not defined